In [2]:
import sys
sys.path.append("..")
from utilities import *

result = transcribe("../audio/example.mp3", "medium.en")

100%|█████████████████████████████████████| 1.42G/1.42G [02:52<00:00, 8.88MiB/s]
100%|██████████| 320960/320960 [09:35<00:00, 557.78frames/s]


### Chapterization
Feed segments to Llama-70b to generate chapter titles.

In [3]:
# Segment the transcript on all pauses
segments = chunk_on_pause(result['segments'], -1, 250, 1500)

# Create Chapter titles for segments
chapters = chapterize(segments[:5])

for c in chapters:
    print(c["title"])
    print(c["time"])
    print(c["metadata"], "\n")
    print(c["text"], "\n\n")

"Introduction to Computer Science and Programming"
[00:00:00,000 --> 00:06:07,000]
(Chunk 1, 1167 words, 0:06:07.120000 mins) 

 The following content is provided under a Creative Commons license. Your support will help MIT OpenCourseWare continue to offer high quality educational resources for free. To make a donation or view additional materials from hundreds of MIT courses, visit MIT OpenCourseWare at ocw.mit.edu. PROFESSOR ERIC GRIMSON Good morning. Try it again. Good morning. Thank you. This is 600, also known as Introduction to Computer Science and Programming. My name is Eric Grimson. Together, Professor John Guttag over here. We're going to be lecturing the course this term. I want to give you a heads up. You're getting some serious firepower this term. John was department head for 10 years, felt like a century. And in course 6, I'm the current department head in course 6. John's been lecturing for 30 years, roughly. All right. I'm the young guy. I've only been lecturing for 25

### Note Generation
Feed the chapters back to Llama-70b to create useful notes from the transcript.

In [10]:
# q = "Imagine you are attending the lecture and tasked with refining the following excerpt into notes. Your goal is to streamline the content by eliminating redundancies and extraneous details. As you organize the notes, feel free to create subheadings where necessary. Your focus should be on retaining crucial information.\n\n"
q = "Write structured notes ONLY in bulletpoint form using the following excerpt of a lecture transcript:\n"

for c in chapters:
    content = '\n'.join([c['title'], c['metadata'], c['text']])
    
    notes = query(q, content)
    filtered ='\n'.join([line for line in notes.split('\n') if line.strip() and line.strip()[0] in ('*', '+', '-')])
    
    print('\n', filtered, '\n')


 * Goals of the course:
	+ Help freshmen and sophomores interested in majoring in Course 6 to get an easy entry into the department.
	+ Help students who don't plan to major in Course 6 feel justifiably confident in their ability to write and read small pieces of code.
	+ Give all students an understanding of the role computation can and cannot play in tackling technical problems.
	+ Position all students to easily compete for jobs, internships, and other opportunities.
* Skills we want you to acquire:
	+ Use the basic tools of computational thinking to write small-scale programs.
	+ Use a vocabulary of computational tools to understand programs written by others.
	+ Understand the fundamental capabilities and limitations of computation, and the costs associated with them. 


 * The course will teach students how to read and write in a computational framework.
* Students will learn how to map scientific problems into a computational framework.
* The course will cover computation, how 